**Approach 1: Stack 3 layers of LSTM - consumed 
95% of the time and efforts, outputs were
not satisfactory - models only as good as the data**

**Approach 2: Transformer based models - simpler, faster, reliable**

**Download the text files from kaggle**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
!pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (2).json


{'kaggle.json': b'{"username":"ashishkumardas1025","key":"14651784e9015469581d00fee636902a"}'}

In [5]:
! mkdir ~/.kaggle1
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle1’: File exists


In [6]:
! kaggle datasets download pariza/bbc-news-summary

  0% 0.00/8.91M [00:00<?, ?B/s]
100% 8.91M/8.91M [00:00<00:00, 109MB/s]


**Extracting folders - News Articles, Summaries**

In [7]:
from zipfile import ZipFile 
  
# specifying the name of the zip file
file = "bbc-news-summary.zip"
  
# open the zip file in read mode
with ZipFile(file, 'r') as zip: 
    # extract all files to another directory
    zip.extractall()
    zip.close()

In [8]:
## We make a Dataframe from Data available in text format 

dictionary={'contents':[],'summaries':[]}
for i in range(1,511):
    if i<10:
        f=open(f'/content/BBC News Summary/News Articles/business/00{i}.txt','r')
        g=open(f'/content/BBC News Summary/Summaries/business/00{i}.txt','r')

    elif 9<i<100:
        f=open(f'/content/BBC News Summary/News Articles/business/0{i}.txt','r')
        g=open(f'/content/BBC News Summary/Summaries/business/0{i}.txt','r')

    else:
        f=open(f'/content/BBC News Summary/News Articles/business/{i}.txt','r')
        g=open(f'/content/BBC News Summary/Summaries/business/{i}.txt','r')
    f.readline()           ## Ignore the heading
    content,summary=f.read(),g.read()
    dictionary['contents'].append(content)
    dictionary['summaries'].append(summary)
    f.close()
    g.close()
        
import pandas as pd
dataframe=pd.DataFrame(dictionary)
dataframe.head(5)

,contents,summaries
0,\nQuarterly profits at US media giant TimeWarn...,TimeWarner said fourth quarter sales rose 2% t...
1,\nThe dollar has hit its highest level against...,The dollar has hit its highest level against t...
2,\nThe owners of embattled Russian oil giant Yu...,Yukos' owner Menatep Group says it will ask Ro...
3,\nBritish Airways has blamed high fuel prices ...,"Rod Eddington, BA's chief executive, said the ..."
4,\nShares in UK drinks and food firm Allied Dom...,Pernod has reduced the debt it took on to fund...


In [9]:
!pip install transformers
!pip install sentencepiece
!pip install syllables

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 59.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 69.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 25.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 939.3/939.3 KB 53.0 MB/s eta 0:00:00
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 6.0.0
    Uninstalling importlib-metadata-6.0.0:
      Successfully uninstalled importlib-metadata-6.0.0


In [10]:
import pandas as pd
import string
import re
import nltk
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()
from textblob import TextBlob, Word
import nltk                  
import ssl 
import syllables

#### The most important problem at this point is Cleaning The Data

1.   Convert to lowercase
2.   Remove Unicode characters
3.   Stop words removal **
4.   Stemming/Lemmatization

In [11]:
## STOPWORDS in Spacy 


stop_words=['those', 'on', 'own', '’ve', 'yourselves', 'around', 'between', 'four', 'been', 'alone', 'off', 'am', 
            'then', 'other', 'can', 'regarding', 'hereafter', 'front', 'too', 'used', 'wherein', '‘ll', 'doing', 
            'everything', 'up', 'onto', 'never', 'either', 'how', 'before', 'anyway', 'since', 'through', 'amount', 
            'now', 'he', 'was', 'have', 'into', 'because', 'therefore', 'they', 'even', 'whom', 'it', 
            'see', 'somewhere', 'thereupon', 'nothing', 'whereas', 'much', 'whenever', 'seem', 'until', 'whereby', 
            'at', 'also', 'some', 'last', 'than', 'get', 'already', 'our', 'once', 'will', 'noone', "'m", 'that', 
            'what', 'thus', 'no', 'myself', 'out', 'next', 'whatever', 'although', 'though', 'which', 'would', 
            'therein', 'nor', 'somehow', 'whereupon', 'besides', 'whoever', 'ourselves', 'few', 'did', 'without', 
            'third', 'anything', 'twelve', 'against', 'while', 'twenty', 'if', 'however', 'herself', 'when', 'may', 
            'ours', 'six', 'done', 'seems', 'else', 'call', 'perhaps', 'had', 'nevertheless', 'where', 'otherwise',
            'still', 'within', 'its', 'for', 'together', 'elsewhere', 'throughout', 'of', 'others', 'show', '’s', 
            'anywhere', 'anyhow', 'as', 'are', 'the', 'hence', 'something', 'hereby', 'nowhere', 'latterly', 'say', 
            'does', 'neither', 'his', 'go', 'forty', 'put', 'their', 'by', 'namely', 'could', 'five', 'unless', 
            'itself', 'is', 'nine', 'whereafter', 'down', 'bottom', 'thereby', 'such', 'both', 'she', 'become', 
            'whole', 'who', 'yourself', 'every', 'thru', 'except', 'very', 'several', 'among', 'being', 'be', 'mine', 
            'further', 'n‘t', 'here', 'during', 'why', 'with', 'just', "'s", 'becomes', '’ll', 'about', 'a', 'using', 
            'seeming', "'d", "'ll", "'re", 'due', 'wherever', 'beforehand', 'fifty', 'becoming', 'might', 'amongst', 
            'my', 'empty', 'thence', 'thereafter', 'almost', 'least', 'someone', 'often', 'from', 'keep', 'him', 
            'or','‘m', 'top', 'her', 'nobody', 'sometime', 'across', '‘s', '’re', 'hundred', 'only', 'via', 'name', 
            'eight','three', 'back', 'to', 'all', 'became', 'move', 'me', 'we', 'formerly', 'so', 'i', 'whence', 
             'under', 'always', 'himself', 'in', 'herein', 'more', 'after', 'themselves', 'you', 'above', 'sixty', 
            'them', 'your', 'made', 'indeed', 'most', 'everywhere', 'fifteen', 'but', 'must', 'along', 'beside', 
            'hers', 
            'side', 'former', 'anyone', 'full', 'has', 'yours', 'whose', 'behind', 'please', 'ten', 'seemed', 
            'sometimes', 'should', 'over', 'take', 'each', 'same', 'rather', 'really', 'latter', 'and', 'ca', 
            'hereupon', 'part', 'per', 'eleven', 'ever', '‘re', 'enough', "n't", 'again', '‘d', 'us', 'yet', 
            'moreover', 'mostly', 'one', 'meanwhile', 'whither', 'there', 'toward', '’m', "'ve", '’d', 'give', 'do', 
            'an', 'quite', 'these', 'everyone', 'towards', 'this', 'cannot', 'afterwards', 'beyond', 'make', 'were', 
            'whether', 'well', 'another', 'below', 'first', 'upon', 'any', 'none', 'many', 'serious', 'various', 
            're', 
            'two', 'less', '‘ve']


## Punctuations that exists in python

import string
punc=string.punctuation

#import re
#import nltk
#from nltk.stem.snowball import SnowballStemmer  ## Snowball Stemmer is preferred over 
                                                ## Porter Stemmer & Lancaster Stemmer(aggresive one)
#from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()
#from textblob import TextBlob, Word

In [12]:
#import nltk                  ## This particular block of code just helps us to download some
#import ssl                   ## necessary tools which wasnt getting downloaded in simple manner

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [13]:
def clean_text(text):
    text=text.strip()    ## stripping white spaces at the end and front
    text=text.lower()    ## Lowering every character
    text=text.encode('ascii', 'ignore').decode()   ## Removing Unicode Characters
    
    ## Let's remove all the stopwords from the text(sentence)
    text_list=text.split()   ## Double or More spaces in texts will be handled at this point
    stopwords_removed_list=[]
    '''for i in text_list:
        if i not in stop_words:
            temp_var=''
            for j in i:
                if j not in punc:
                    temp_var+=str(j)
            if temp_var not in stop_words:    
                stopwords_removed_list.append(temp_var)
    text=' '.join(stopwords_removed_list)  ## Sentence after stop words removal and punctuation removal

    
    
    ## Having Done with points 1,2,3 in Text cleaning steps we progress to do stemming & Lemmatization'''
    
    ''' stemmer= SnowballStemmer("english", ignore_stopwords=True) ## stopwords are already removed so 
                                                                   ##second parameter
                                                                   ## may not have any added value

    word_list=text.split()        ## Basic method to tokenize a sentence
    
    final_word_list=[snowBallStemmer.stem(word) for word in word_list]  ## Stemming every word
    
    text=' '.join(final_word_list) '''   ## Will use this part of code if needed 
    
    ## It seems that lemmatization is better way to normalize texts
    
    word_list=text.split()
    final=[]
    for word in word_list:
        w=lemmatizer.lemmatize(word)
        final.append(w)
    text=' '.join(final)
    return text

In [14]:
lemmatizer.lemmatize("answers")

'answer'

In [15]:
del zip

In [16]:
## Recreating Dataframe with cleaned entries and after normalizing texts

content_list=dataframe['contents'].to_list()
summaries_list=dataframe['summaries'].to_list()
my_dict={'Contents':[],'Summaries':[]}
for i,j in zip(content_list,summaries_list):
    my_dict['Contents'],my_dict['Summaries']=my_dict['Contents']+[clean_text(i)],my_dict['Summaries']+[j]

DF=pd.DataFrame(my_dict)
DF.head()

,Contents,Summaries
0,quarterly profit at u medium giant timewarner ...,TimeWarner said fourth quarter sales rose 2% t...
1,the dollar ha hit it highest level against the...,The dollar has hit its highest level against t...
2,the owner of embattled russian oil giant yukos...,Yukos' owner Menatep Group says it will ask Ro...
3,british airway ha blamed high fuel price for a...,"Rod Eddington, BA's chief executive, said the ..."
4,share in uk drink and food firm allied domecq ...,Pernod has reduced the debt it took on to fund...


In [17]:
## The models takes two parameters min_length and max_length

summ_dict=dict()
summary_list=DF['Summaries'].to_list()
summ_list=[len(x) for x in summary_list]
max_length=(sum(summ_list)//len(summ_list))  ## Average summary length stored in parameter max_length
min_length=min(summ_list)
print(min_length,max_length)

content_list=DF['Contents'].to_list()
cont_list=[len(i) for i in content_list]
print(int(0.333333333*(sum(cont_list)//len(cont_list))))

## We use the fact that length of summary of an article  will lie between
## minimum length of all summaries given in our dataset and one-third of average length of the Contents 

## IN OTHER WORDS LENGTH OF SUMMARY OF AN ARTICLE LIES IN [243,638]

243 865
638


In [18]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import torch

In [19]:
!pip install datasets
from datasets import load_metric
!pip install nltk rouge_score

from rouge_score import rouge_scorer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 18.5 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=1660fadaf4c6737b6c8cbc356000452b0f43e0cbe570b5b65d552a88052e9292
  Stored in directory: /root/.cache/pip/wheels/24/55/6f/ebfc4cb176d1c9665da4e306e1705496206d08215c1acd9dde
Successfully built rouge_score


**Pegasus - 568M parameters - pretrained on C4(750GB) and HugeNews(3.8TB) - GSG - principal gap selection**

In [26]:
model_name = "google/pegasus-xsum"
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)

## We use a Loop over the dataframe to summarize every content

summarized_contents=[]
frac = 0.2

for i in range(10):
  article=DF['Contents'][i]
  max_len = frac*(len(article.split()))
  inputs = tokenizer(article, truncation = True,padding='longest', return_tensors="pt").to(device)
  inp_dec = model.generate(**inputs, max_length = max_len)
  summarized = tokenizer.batch_decode(inp_dec, skip_special_tokens=True)
  summarized_contents.append(summarized)

In [21]:
from transformers import pipeline

In [29]:
summarization = pipeline("summarization")
summarized_contents_=[]
for i in range(10):
  summary_text = summarization(DF['Contents'][i])[0]
  summarized_contents_.append(summary_text)

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [24]:
summarized_contents_[1]['summary_text']

' dollar hits highest level against euro in almost three month . Federal reserve head says trade deficit is set to stabilise . Greenback tumbled on the back of worse-than-expected u job data . Concern about the deficit concern about china do, however, remain .'

In [22]:
import numpy as np

In [30]:

## We have calculated the metric for the first 5 rows of our dataset
scorer = rouge_scorer.RougeScorer(['rouge1','rouge2','rougeL'], use_stemmer=True)
precision=[]
precision_=[]
recall=[]
recall_=[]
f1=[]
f1_=[]
for i in range(10):
  score = scorer.score(summarized_contents[i][0],DF['Summaries'][0])
  precision.append(score['rouge2'].precision)
  recall.append(score['rouge2'].recall)
  f1.append(score['rouge2'].fmeasure)

for i in range(10):
  score = scorer.score(summarized_contents_[i]['summary_text'],DF['Summaries'][0])
  precision_.append(score['rouge2'].precision)
  recall_.append(score['rouge2'].recall)
  f1_.append(score['rouge2'].fmeasure)


metric_df = pd.DataFrame({'precision' : [np.mean(precision), np.mean(precision_)],
                          'recall' : [np.mean(recall), np.mean(recall_)],
                          'f1' : [np.mean(f1), np.mean(f1_)]})

In [31]:
metric_df

,precision,recall,f1
0,0.005263,0.049848,0.009466
1,0.028947,0.112679,0.045972


In [32]:
inputs

{'input_ids': tensor([[  114,  4911,   657,  1844, 29752,   109,   531,   131,   116,  1715,
          9446,   301,   113,  3693,   164,   109,  1298,   113,  6003,  8886,
           174,  6723,   165,   141,   142,  3455,  1462,   107,   109,  1806,
           118, 92107, 10970,   143, 31788, 10970,   158,   233,  3252,   141,
           109, 64545, 41604,  2633,   115,  8094,   233, 21445,  7752,   115,
           114, 16392,  1057,   107,   109,  1462,   115, 63409,   374,   120,
           109,   437,   256,   146,   129,  1457,   365,  2043,  1857,   121,
         24884,  5009, 37668,  2564,   107,   790,   109,  6462,   195, 20913,
          8330,   456,   108,   110, 63640,   920, 10152, 25856,  9446,   108,
           110, 15548, 41062,  9446,   108, 19683, 62354,   456,   111,  2799,
           111, 75606,   661,   107,   115,   126,   437,   108,   109,   657,
          4620,  9446,  1419, 25735, 21628,   476,   112,   815,  6030,   108,
          4635, 10686,   122,  1546,  

In [33]:
summarized_contents[0]

['Timewarner has reported a sharp rise in fourth quarter profit, helped by a one-off gain from its stake in google.']

In [34]:
DF['Contents'][0]

'quarterly profit at u medium giant timewarner jumped 76% to $1.13bn (600m) for the three month to december, from $639m year-earlier. the firm, which is now one of the biggest investor in google, benefited from sale of high-speed internet connection and higher advert sales. timewarner said fourth quarter sale rose 2% to $11.1bn from $10.9bn. it profit were buoyed by one-off gain which offset a profit dip at warner bros, and le user for aol. time warner said on friday that it now owns 8% of search-engine google. but it own internet business, aol, had ha mixed fortunes. it lost 464,000 subscriber in the fourth quarter profit were lower than in the preceding three quarters. however, the company said aol\'s underlying profit before exceptional item rose 8% on the back of stronger internet advertising revenues. it hope to increase subscriber by offering the online service free to timewarner internet customer and will try to sign up aol\'s existing customer for high-speed broadband. timewarn

**Next Steps:** We will use newspaper library to webscrape website's article (allowing webscrapping) and summarize it i.e., URL based Summarization and will deploy the pegasus model using Streamlit platform.

In [35]:
!pip install protobuf==3.20.3
!pip install streamlit -q
!pip install transformers

!pip install newspaper3k
!pip install sentencepiece
!pip install beautifulsoup4
!pip install google

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.6
    Uninstalling protobuf-3.19.6:
      Successfully uninstalled protobuf-3.19.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.9.2 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.
tensorboard 2.9.1 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 88.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 KB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.0/184.0 KB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [36]:
%%writefile summarize.py
import streamlit as st
from newspaper import Article
import nltk
from math import floor
from PIL import Image
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import torch
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('omw-1.4')
st.image("https://eecsnews.engin.umich.edu/wp-content/uploads/sites/2/2021/03/summarization-featured.jpg")
#st.text_input("FINANCIAL TEXT SUMMARIZATION", kwargs=None, placeholder=None, disabled=True)

url = st.text_input(
        "input the URL👇")
if url:  
  article = Article(url)
  article.download()
  article.parse()
  #article.text
  def read_text(path):
    with open(path,'r') as f:
      lines=f.readlines()
    return lines[0]
  model_name = "google/pegasus-xsum"
  device = "cuda" if torch.cuda.is_available() else "cpu"
  tokenizer = PegasusTokenizer.from_pretrained(model_name)
  model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)
  article=article.text
  print(article)
  max_len = floor(0.2*(len(article.split())))
  inputs = tokenizer(article, truncation = True,padding='longest', return_tensors="pt").to(device)
  inp_dec = model.generate(**inputs, max_length = max_len)
  summarized = tokenizer.batch_decode(inp_dec, skip_special_tokens=True)
  summarized = " ".join(summarized)
  st.write(summarized)


Writing summarize.py


In [37]:
%%writefile summarize.py
import streamlit as st
from newspaper import Article
import nltk
from math import floor
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import torch
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('omw-1.4')
import string
import syllables
import math
import random
punc=string.punctuation

def read_text(path):
  with open(path,'r') as f:
    lines=f.readlines()
  return lines[0]

def mod_sen(sentence):
  modi_sen=''
  for i in sentence:
    if i in punc:
      modi_sen+='.'
    else:
      modi_sen+=i
  return modi_sen

def flesch_kincaid(text):
    text=mod_sen(text)
    total_sentences,total_words=len(text.split('.'))-1,(len(text.split()))+text.count('.')-1
    sentences=text.split('.')
    syl_count=0
    for i in sentences:
      l=i.split()
      for j in l:
        syl_count+=syllables.estimate(j)
    print(syl_count)
    reading_ease=(210.835-(7.015*(total_words/total_sentences))-(90.6*(syl_count/total_words)))
    reading_grade=((0.39*(total_words/total_sentences))+(11.8*(syl_count/total_words))-15.59)
    if reading_ease>0:
      return math.floor(reading_ease)
    else:
      epsilon=(random.random())/100
      return epsilon

def fog_index(text):
  text=mod_sen(text)
  total_sentences,total_words=len(text.split('.'))-1,(len(text.split()))+text.count('.')-1
  sentences=text.split('.')
  complexity=0
  for i in sentences:
    l=i.split()
    for j in l:
      if syllables.estimate(j)>2:
        complexity+=1
  return (0.7*((total_words/total_sentences)+100*(complexity/total_words)))

st.image("https://eecsnews.engin.umich.edu/wp-content/uploads/sites/2/2021/03/summarization-featured.jpg")
url = st.text_input(
        "input url")
if url:  
  article = Article(url)
  article.download()
  article.parse()


  model_name = "google/pegasus-xsum"
  device = "cuda" if torch.cuda.is_available() else "cpu"
  tokenizer = PegasusTokenizer.from_pretrained(model_name)
  model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)
  article=article.text
  t=mod_sen(article)
  l=t.split()
  o_standard_reading_time=len(l)/80
  max_len = floor(0.2*(len(article.split())))
  inputs = tokenizer(article, truncation = True,padding='longest', return_tensors="pt").to(device)
  inp_dec = model.generate(**inputs, max_length = max_len)
  summarized = tokenizer.batch_decode(inp_dec, skip_special_tokens=True)
  summarized = " ".join(summarized)
  s_standard_reading_time=len(summarized.split())/80
  st.write(summarized)

  st.write(f'The Readability score of the original article is {flesch_kincaid(article)}')
  st.write(f'The Reading time of the original text is {o_standard_reading_time} mins')
  st.write(f'The Readability score of the summarized text is {flesch_kincaid(summarized)}')
  st.write(f'The Reading time of the summarized text is {s_standard_reading_time} mins')

Overwriting summarize.py


In [38]:
!streamlit run summarize.py & npx localtunnel --port 8501

[########..........] / extract:openurl: sill extract string-width@^4.2.0 extrac

npx: installed 22 in 3.861s

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://34.85.254.224:8501

your url is: https://late-symbols-carry-34-85-254-224.loca.lt
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
447
47
  